# NAME: Grey Harris
# ID: 97774899
# COURSE: Data301
# LAB: 4

# Imports and Helper Functions

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

import pyspark, os
import math
from math import sqrt
from math import log
from random import seed
from random import randint
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass


#also include this short helper function for use later in this lab
def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 49.8 MB/s 


In [3]:
# start the Spark context
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
import os
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

import pyspark
from pyspark import SparkConf, SparkContext

try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

    

import unittest
Test = unittest.TestCase()

In [4]:
!gdown 'https://drive.google.com/uc?export=download&confirm=t&id=1Ijyh14a0Lh9sjwQUR6PE1TB2phjAZP4P'

Downloading...
From: https://drive.google.com/uc?export=download&confirm=t&id=1Ijyh14a0Lh9sjwQUR6PE1TB2phjAZP4P
To: /content/browsing.txt
100% 3.46M/3.46M [00:00<00:00, 197MB/s]


#QUESTION ONE

###Q 1A

In [5]:
def line_to_list(word_list):
  """takes a string containing one or more space seperated elements and returns them as a list"""
  word_list = word_list.strip().split()
  listy = []
  for potato, i in enumerate(word_list):
    for potatoe, j in enumerate(word_list):
      if potatoe < potato:
        listy.append(tuple(sorted((i, j))))
  return listy

file = sc.textFile('browsing.txt')

pairsRDD = file.flatMap(lambda line: (line_to_list(line)))

pairOccurancesRDD = pairsRDD.map(lambda pair: (pair, 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda pair: pair[1], 0)

print("Should match")
print("[(('DAI62779', 'ELE17451'), 1592), (('FRO40251', 'SNA80324'), 1412), (('DAI75645', 'FRO40251'), 1254), (('FRO40251', 'GRO85051'), 1213), (('DAI62779', 'GRO73461'), 1139)]")
dbg(pairOccurancesRDD.take(5))

#dbg(pairOccurancesRDD.count())
#dbg(pairsRDD.count())

Should match
[(('DAI62779', 'ELE17451'), 1592), (('FRO40251', 'SNA80324'), 1412), (('DAI75645', 'FRO40251'), 1254), (('FRO40251', 'GRO85051'), 1213), (('DAI62779', 'GRO73461'), 1139)]
[(('DAI62779', 'ELE17451'), 1592), (('FRO40251', 'SNA80324'), 1412), (('DAI75645', 'FRO40251'), 1254), (('FRO40251', 'GRO85051'), 1213), (('DAI62779', 'GRO73461'), 1139)]


###Q 1B

In [6]:
def line_to_list_two(word_list):
  """takes a string containing one or more space seperated elements and returns them as a list"""
  word_list = word_list.strip().split()
  listy = []
  for potato, i in enumerate(word_list):
    for potatoe, j in enumerate(word_list):
      if potatoe < potato and (i in broadcastedRDD.value and j in broadcastedRDD.value):
        listy.append(tuple(sorted((i, j))))
  return listy


def filter_values(RDD, filter_value=100):
  """filters out values under 100 by default but can take second argument of a positive integer value to filter by"""
  resultRDD = RDD.filter(lambda x: x[1] >= filter_value)
  return resultRDD

def map_and_collect(RDD):
  """removes counts from mapped tuples returning a map of just the items"""
  #mappedRDD = RDD.map(lambda y: y[0])
  #resultRDD = mappedRDD.collect()
  resultRDD = RDD.collectAsMap()
  return resultRDD

itemsRDD = file.flatMap(lambda string: string.strip().split(" "))
dbg(file)
print(itemsRDD)
dbg(itemsRDD)

uniqueItemsRDD = itemsRDD.map(lambda item: (item, 1)).reduceByKey(lambda x,y:x+y)
#print(uniqueItemsRDD)
#dbg(uniqueItemsRDD)

print("Below should match ")
dbg(uniqueItemsRDD.takeOrdered(5,lambda x: -x[1]))
print("[('DAI62779', 6667), ('FRO40251', 3881), ('ELE17451', 3875), ('GRO73461', 3602), ('SNA80324', 3044)]")

filteredRDD = map_and_collect(filter_values(uniqueItemsRDD))
broadcastedRDD = sc.broadcast(filteredRDD)

#print("\nBelow should match above but missing the counts")
#print("[('DAI62779', 6667), ('FRO40251', 3881), ('ELE17451', 3875), ('GRO73461', 3602), ('SNA80324', 3044)]")
#dbg(filter_values(uniqueItemsRDD).takeOrdered(5,lambda x: -x[1]))
#print(broadcastedRDD.value[:5])

wordPairsRDD = file.flatMap(lambda string: line_to_list_two(string))
pairsResultsRDD = wordPairsRDD.map(lambda item: (item, 1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[1], 0)
#dbg(pairsResultsRDD.count())
#dbg(wordPairsRDD)
#dbg(parisResultsRDD)

print("\nBelow should match, 1b")
print(pairsResultsRDD.takeOrdered(5,lambda x: -x[1]))
print("[(('DAI62779', 'ELE17451'), 1592), (('FRO40251', 'SNA80324'), 1412), (('DAI75645', 'FRO40251'), 1254), (('FRO40251', 'GRO85051'), 1213), (('DAI62779', 'GRO73461'), 1139)]")

#dbg(pairsResultsRDD.count())

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ', 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ', 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ', 'ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ', 'ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ', 'ELE17451 GRO73461 DAI22896 SNA99873 FRO18919 DAI50921 SNA80192 GRO75578 ', 'ELE17451 ELE59935 FRO18919 ELE23393 SNA80192 SNA85662 SNA91554 DAI22177 ', 'ELE17451 SNA69641 FRO18919 SNA90258 ELE28573 ELE11375 DAI14125 FRO78087 ', 'ELE17451 GRO73461 DAI22896 SNA80192 SNA85662 SNA90258 DAI46755 FRO81176 ELE66810 DAI49199 DAI91535 GRO94758 ELE94711 DAI22177 ', 'ELE17451 SNA69641 DAI91535 GRO94758 GRO99222 FRO76833 FRO81176 SNA80192 DAI54690 ELE37798 GRO56989 ', 'ELE17451 GRO73461 DAI22896 GRO99222 SNA47306 GRO36567 ELE82555 SNA17715 SNA94781 DAI87514 GRO48282 GRO12935 SNA55952 DAI93692 DAI92253 FRO82427 ELE26917 DAI22177 ', 'GRO99222 DAI48891 

In [7]:
dbg(broadcastedRDD)

#QUESTION 2


### Q2 A

In [8]:
# // broadcast = broadcastedRDD

def division(x, y, total):
  divisor = broadcastedRDD.value[x]
  answer = ((x, y), total/divisor)
  return answer

pairsResultsRDD.map(lambda x: ((x[0][1], x[0][0])))
#dbg(pairsResultsRDD)

xyx = pairsResultsRDD.map(lambda x: division(x[0][0], x[0][1], x[1]))
yxy = pairsResultsRDD.map(lambda x: division(x[0][1], x[0][0], x[1]))
confidenceScores = xyx.union(yxy)

print("Below should match, 2a (I just ignored the rounding I assume its okay)\n")
dbg(confidenceScores.takeOrdered(5,lambda x: -x[1]))
print("[(('DAI93865', 'FRO40251'), 1.0), (('GRO85051', 'FRO40251'), 0.9991762767710051), (('GRO38636', 'FRO40251'), 0.9906542056074765), (('ELE12951', 'FRO40251'), 0.9905660377358491), (('DAI88079', 'FRO40251'), 0.9867256637168142)]")

Below should match, 2a (I just ignored the rounding I assume its okay)

[(('DAI93865', 'FRO40251'), 1.0), (('GRO85051', 'FRO40251'), 0.999176276771005), (('GRO38636', 'FRO40251'), 0.9906542056074766), (('ELE12951', 'FRO40251'), 0.9905660377358491), (('DAI88079', 'FRO40251'), 0.9867256637168141)]
[(('DAI93865', 'FRO40251'), 1.0), (('GRO85051', 'FRO40251'), 0.9991762767710051), (('GRO38636', 'FRO40251'), 0.9906542056074765), (('ELE12951', 'FRO40251'), 0.9905660377358491), (('DAI88079', 'FRO40251'), 0.9867256637168142)]


###Q2 B

In [9]:
basket_case = file.map(lambda tuple_key: tuple_key.split(" ,")).count()

def calculate(my_tuple, number):
  tuple_key = my_tuple[0][1]
  return((my_tuple[0], number - (broadcastedRDD.value[tuple_key]/basket_case)))

score = confidenceScores.map(lambda x: calculate(x, x[1]))

print("\nBelow should match, also 2b")
dbg(score.takeOrdered(5,lambda x: -x[1]))
print("[(('DAI43868', 'SNA82528'), 0.9538739086342056), (('DAI93865', 'FRO40251'), 0.8752130156586605), (('GRO85051', 'FRO40251'), 0.8743892924296656), (('GRO38636', 'FRO40251'), 0.865867221266137), (('ELE12951', 'FRO40251'), 0.8657790533945096)]")


Below should match, also 2b
[(('DAI43868', 'SNA82528'), 0.9538739086342057), (('DAI93865', 'FRO40251'), 0.8752130156586605), (('GRO85051', 'FRO40251'), 0.8743892924296655), (('GRO38636', 'FRO40251'), 0.8658672212661371), (('ELE12951', 'FRO40251'), 0.8657790533945096)]
[(('DAI43868', 'SNA82528'), 0.9538739086342056), (('DAI93865', 'FRO40251'), 0.8752130156586605), (('GRO85051', 'FRO40251'), 0.8743892924296656), (('GRO38636', 'FRO40251'), 0.865867221266137), (('ELE12951', 'FRO40251'), 0.8657790533945096)]
